In [1]:
# STANDARD LIBRARY IMPORTS
import os
import datetime
import glob

# THIRD PARTY IMPORTS
import praw
import pandas as pd

# PROJECT IMPORTS
from setup import reddit
from settings import *

subreddit = reddit.subreddit(TARGET_SUBREDDIT)

Loaded 0 users who have opted out from follow-ups.
Loaded 0 users who have opted out from reminders.


Version 7.5.0 of praw is outdated. Version 7.6.0 was released Tuesday May 10, 2022.


In [2]:
# load most recent 2 weeks worth of awards
recent_awardsfiles = sorted(glob.glob( os.path.join(PATH_TO_STORE, 'awards*.csv') ), reverse=True)

# last two files
df = pd.concat([pd.read_csv(f) for f in recent_awardsfiles[:2]])

In [3]:
# filter to recent WEEKLY_STICKY_THREAD_TIME_PERIOD weeks
recency_threshold = datetime.datetime.utcnow() - datetime.timedelta(weeks=WEEKLY_STICKY_THREAD_TIME_PERIOD)
df = df.loc[ df.awarding_timestamp > int(recency_threshold.timestamp()) ,:]

In [20]:
# compute top awardees
awardees = df.groupby('awarded_author').agg({'awarding_id':'count'}).iloc[:,0].sort_values(ascending=False)

# include more awardees in case of ties
awardees = awardees.loc[ awardees >= awardees.iloc[NUM_TOP_AWARDEES-1] ]

In [21]:
awardees

awarded_author
kenerling            6
theHanMan62          3
Superman_Dam_Fool    2
chuckman13           2
BlurredOrange        2
bmbphotos            2
hippobiscuit         2
stygian_dj           2
Kudzupatch           2
Name: awarding_id, dtype: int64

In [26]:
submissions = df.groupby('submission_id').agg({'awarding_id':'count'}).iloc[:,0].sort_values(ascending=False).iloc[:NUM_TOP_THREADS]

In [27]:
submissions

submission_id
t3_v1u9zc    13
t3_uymggw    11
t3_v17av2    10
Name: awarding_id, dtype: int64

In [36]:
# fetch submission titles and urls from the api

def format_submission_link(prefixed_id):
    submission = reddit.submission(id=prefixed_id[3:])
    
    try:
        title = submission.title
        link  = submission.permalink
        return f'[{title}](https://reddit.com{link})'
    except:
        return None
    
submissions_formatted = map(format_submission_link, submissions.index)

In [37]:
submissions.index = submissions_formatted

In [39]:
submissions = submissions.loc[submissions.index.notnull()]

In [40]:
submissions

[Does something feel off with this photo or is it just me?](https://reddit.com/r/photocritique/comments/v1u9zc/does_something_feel_off_with_this_photo_or_is_it/)    13
[2 people and louvre](https://reddit.com/r/photocritique/comments/uymggw/2_people_and_louvre/)                                                                       11
[ITAP of the sea (Is this picture overly edited?)](https://reddit.com/r/photocritique/comments/v17av2/itap_of_the_sea_is_this_picture_overly_edited/)                10
Name: awarding_id, dtype: int64